<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Mathematics Basics

**With Python**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Imports & Configurations

In [ ]:
!git clone https://github.com/tpq-classes/mathematics_basics.git
import sys
sys.path.append('mathematics_basics')


In [ ]:
import math
import random

In [ ]:
from pylab import plt
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

## Helper Functions

In [ ]:
def mean(rn):
    return sum(rn) / len(rn)

In [ ]:
def var(rn):
    mu = mean(rn)
    return sum([(n - mu) ** 2 for n in rn]) / len(rn)

In [ ]:
def std(rn):
    return math.sqrt(var(rn))

In [ ]:
def cum_sum(rn):
    s = 0
    cs = list()
    for n in rn:
        s += n
        cs.append(s)
    return cs

## Financial Stochastic Processes

Applying the Monte Carlo method to simulate typical financial processes.

### Geometric Brownian Motion

Assume the following parameters for the Black-Scholes-Merton (1973) stock price model (geometric Brownian motion) as follows:

* $S_0 = 100$, initial price
* $\sigma = 0.2$, volatility (percent in digits)
* $r = 0.06$, constant short rate
* $T = 1$, time horizon (in year fractions)
* $S_T$, future price (log-normally distributed)

It holds $\frac{dS_t}{S_t} = r dt + \sigma dZ_t$, with $Z_t$ a standard brownian motion (random walk).

In discrete time, it holds $S_{t}= S_{t - \Delta t} e^{\left(r - \frac{\sigma^2}{2}\right) \Delta t + \sigma \sqrt{\Delta t} z_t}$, for $t>0, \Delta t > 0$.

See also https://en.wikipedia.org/wiki/Geometric_Brownian_motion.

In [ ]:
S0, sigma, r, T = 100, 0.2, 0.06, 1.0

In [ ]:
M = 50

In [ ]:
dt = T / M
dt

In [ ]:
rn = [(r - sigma ** 2 / 2) * dt + sigma * math.sqrt(dt) * random.gauss(0, 1) for _ in range(M)]

In [ ]:
rn.insert(0, 0)

In [ ]:
rn[:5]

In [ ]:
gbm = [S0]
for n in rn[1:]:
    gbm.append(gbm[-1] * math.exp(n))

In [ ]:
gbm[:5]

In [ ]:
gbm = [S0 * math.exp(n) for n in cum_sum(rn)]

In [ ]:
gbm[:5]

In [ ]:
plt.plot(gbm[:]);

In [ ]:
def simulate_gbm(M, T, r, sigma):
    dt = T / M
    rn = [(r - sigma ** 2 / 2) * dt +
          sigma * math.sqrt(dt) * random.gauss(0, 1)
          for _ in range(M)]
    rn.insert(0, 0)
    gbm = [S0 * math.exp(n) for n in cum_sum(rn)]
    return gbm

In [ ]:
gbm_procs_1 = [simulate_gbm(M, 1, 0.1, 0.2) for _ in range(1000)]

In [ ]:
for proc in gbm_procs_1[:50]:
    plt.plot(proc, 'b--', lw=1)

In [ ]:
gbm_procs_2 = [simulate_gbm(M, 1, 0.2, 0.4) for _ in range(1000)]

In [ ]:
for proc in gbm_procs_2[:50]:
    plt.plot(proc, 'r--', lw=1)
for proc in gbm_procs_1[:50]:
    plt.plot(proc, 'b--', lw=1)

In [ ]:
S1_mean = list()
for t in range(M + 1):
    me = mean([proc[t] for proc in gbm_procs_1])
    S1_mean.append(me)

In [ ]:
S2_mean = list()
for t in range(M + 1):
    me = mean([proc[t] for proc in gbm_procs_2])
    S2_mean.append(me)

In [ ]:
for proc in gbm_procs_2[:25]:
    plt.plot(proc, 'r--', lw=1)
for proc in gbm_procs_1[:25]:
    plt.plot(proc, 'b--', lw=1)
plt.plot(S2_mean, 'r-', lw=3, label='high drift/vol mean')
plt.plot(S1_mean, 'b-', lw=3, label='low drift/vol mean')
plt.legend();

In [ ]:
ST_1 = [proc[-1] for proc in gbm_procs_1]

In [ ]:
ST_2 = [proc[-1] for proc in gbm_procs_2]

In [ ]:
mean(ST_1)

In [ ]:
math.exp(0.1)

In [ ]:
S0 * math.exp(0.1 * T)

In [ ]:
std(ST_1)

In [ ]:
lr_1 = [math.log(ST / S0) for ST in ST_1]

In [ ]:
std(lr_1)  # volatility

In [ ]:
mean(ST_2)

In [ ]:
S0 * math.exp(0.2 * T)

In [ ]:
std(ST_2)

In [ ]:
lr_2 = [math.log(ST / S0) for ST in ST_2]

In [ ]:
std(lr_2)  # volatility

In [ ]:
# plt.axvline?

In [ ]:
plt.hist(ST_2, bins=30, label='high drift/vol')
plt.hist(ST_1, bins=30, label='low drift/vol')
plt.axvline(mean(ST_2), color='r', label='high drift/vol mean')
plt.axvline(mean(ST_1), color='m', label='low drift/vol mean')
plt.legend();

### Jump Diffusion

Assume the Merton (1976) model which adds a jump component to the Black-Scholes-Merton (1973) model:

$ dS_t = (r-r_j)S_t dt + \sigma S_t dZ_t + J_t S_t dN_t$

The meaning of the variables are:

* $S_t$ index level at date $t$
* $r$ constant riskless short rate
* $r_{J}\equiv \lambda \cdot \left(e^{\mu_{J}+\delta^{2}/2}-1\right)$ drift correction for jump to maintain risk neutrality
* $\sigma$ constant volatility of $S$
* $Z_t$ standard Brownian motion
* $J_t$ jump at date $t$ with distribution ...
  * ... $\log(1+J_{t})\approx \mathbf{N}\left(\log(1+\mu_{J})-\frac{\delta^{2}}{2},\delta^{2}\right)$ with ...
  * ... **N** as the cumulative distribution function of a standard normal random variable
* $N_t$ Poisson process with intensity $\lambda$

An Euler discretization scheme is given by:

$S_{t}=S_{t-\Delta t}\left(e^{(r-r_{J}-\sigma^{2}/2)\Delta t+\sigma \sqrt{\Delta t}z^{1}_{t}}+ \left(e^{\mu_{J}+\delta z^{2}_{t}}-1\right)y_{t}\right)$

with $z^1_t, z^2_t$ being standard normally distributed and $y_t$ being Poisson distributed.

See also https://en.wikipedia.org/wiki/Jump_diffusion.

In [ ]:
def poisson(lamb):
    t = 0
    c = -1
    while t < 1:
        n = -1 / lamb * math.log(random.random())
        t += n
        c += 1
    return c

In [ ]:
muj, delta, lamb = -0.5, 0.1, 0.5

In [ ]:
rj = lamb * (math.exp(muj + delta ** 2 / 2) - 1)

In [ ]:
rj

In [ ]:
M = 100

In [ ]:
T

In [ ]:
dt = T / M

In [ ]:
random.seed(1000)

In [ ]:
rn_diff = [(r - rj - sigma ** 2 / 2) * dt +
           sigma * math.sqrt(dt) * random.gauss(0, 1)
           for _ in range(M)]

In [ ]:
rn_jump = [(math.exp(muj + delta * random.gauss(0, 1)) - 1) *
           poisson(lamb * dt) for _ in range(M)]

In [ ]:
rn = [math.exp(d) + j for d, j in zip(rn_diff, rn_jump)]

In [ ]:
rn[:6]

In [ ]:
jd = [S0]
for n in rn:
    jd.append(jd[-1] * n)

In [ ]:
plt.plot(jd);

In [ ]:
def simulate_jd():
    rn_diff = [(r - rj - sigma ** 2 / 2) * dt +
               sigma * math.sqrt(dt) * random.gauss(0, 1)
               for _ in range(M)]
    rn_jump = [(math.exp(muj + delta * random.gauss(0, 1)) - 1) *
               poisson(lamb * dt) for _ in range(M)]
    rn = [math.exp(d) + j for d, j in zip(rn_diff, rn_jump)]
    jd = [S0]
    for n in rn:
        jd.append(jd[-1] * n)
    return jd

In [ ]:
jd_procs = [simulate_jd() for _ in range(10000)]

In [ ]:
for proc in jd_procs[:20]:
    plt.plot(proc, 'b--', lw=1)

In [ ]:
ST = [proc[-1] for proc in jd_procs]

In [ ]:
mean(ST)

In [ ]:
std(ST)

In [ ]:
lr = [math.log(s / S0) for s in ST]

In [ ]:
std(lr)

In [ ]:
mean(lr)

In [ ]:
std(lr)

In [ ]:
plt.hist(ST, bins=50);

In [ ]:
plt.hist(lr, bins=50);

## Europen Option Valuation

Assume the previous stock price model and assume that a European option has a strike price of $K = 100$.

The payoff of a European options is:
* $\max(S - K, 0)$ for a European call option
* $\max(K - S, 0)$ for a European put option

In [ ]:
K = 100

* **stock price**: random phenomenon = nature/markets --> stock price (model)
* **option payoff**: random phenomenon = stock price --> option payoff (model)

#### European Call option

In [ ]:
ST = [proc[-1] for proc in jd_procs]

In [ ]:
CT = [max(s - K, 0) for s in ST]

In [ ]:
CT[:10]

In [ ]:
# Monte Carlo Estimator
C0 = math.exp(-r * T) * mean(CT)  # present value

In [ ]:
C0

In [ ]:
mean(CT)  # future value (estimator)

In [ ]:
RC = [math.log(c / C0 + 0.00001) for c in CT]  # log returns

In [ ]:
var(RC)

In [ ]:
std(RC)

In [ ]:
plt.title('European call option payoff')
plt.hist(CT, bins=50, label='with jumps')
plt.legend();

#### European Put option

In [ ]:
PT = [max(K - s, 0) for s in ST]

In [ ]:
PT[:10]

In [ ]:
# Monte Carlo Estimator
P0 = math.exp(-r * T) * mean(PT)  # present value

In [ ]:
P0

In [ ]:
mean(PT)  # future value (estimator)

In [ ]:
RP = [math.log(p / P0 + 0.00001) for p in PT]  # log returns

In [ ]:
var(RP)

In [ ]:
std(RP)

In [ ]:
plt.title('European put option payoff')
plt.hist(PT, bins=50, label='with jumps')
plt.legend();

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>